In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools as iter
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import seaborn as sn

In [3]:
dataset = pd.read_csv('fake_or_real_news.csv')
x = dataset['text']
y = dataset['label']

In [4]:
dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
dataset.shape

(6335, 4)

In [6]:
dataset.isnull().any()

Unnamed: 0    False
title         False
text          False
label         False
dtype: bool

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [12]:
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train=tfidf_vectorizer.fit_transform(x_train)
tfidf_test=tfidf_vectorizer.transform(x_test)

In [17]:
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.84%


In [18]:
pipeline = Pipeline([('tfidf',TfidfVectorizer(stop_words='english')),
                    ('nbmodel',MultinomialNB())])

In [19]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('nbmodel', MultinomialNB())])

In [20]:
score=pipeline.score(x_test,y_test)
print('accuracy',score)

accuracy 0.8310970797158642


In [21]:
pred = pipeline.predict(x_test)

In [22]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

        FAKE       0.98      0.67      0.80       627
        REAL       0.75      0.99      0.86       640

    accuracy                           0.83      1267
   macro avg       0.87      0.83      0.83      1267
weighted avg       0.87      0.83      0.83      1267



In [23]:
print(confusion_matrix(y_test,pred))

[[420 207]
 [  7 633]]


In [24]:
with open('model.pkl','wb') as handle:
    pickle.dump(pipeline, handle, protocol=pickle.HIGHEST_PROTOCOL)